In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
# import joblib
from datetime import datetime
from collections import Counter
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

In [4]:
import kagglehub

# Download latest version
path = "/kaggle/input/open12"

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/open12


In [9]:
# f*string: 변수를 안에 넣는다. (자주 공통적으로 사용되는 변수는 f 스트링을 써야한다. )
# 데이터 로드
train = pd.read_csv(f'{path}/train.csv')

In [11]:
# Feature & Target 설정
x = train.drop(['ID', '허위매물여부'], axis=1)
y = train['허위매물여부']

In [23]:
from sklearn.impute import KNNImputer
import pandas as pd

# KNN Imputer 설정 (n_neighbors=5)
knn_imputer = KNNImputer(n_neighbors=5)

# 결측치를 채울 열 리스트
columns_fill_knn = ['해당층', '총층', '전용면적', '방수', '욕실수', '총주차대수']

# KNNImputer를 사용하여 결측치 채우기
x[columns_fill_knn] = knn_imputer.fit_transform(x[columns_fill_knn])

# KNNImputer 후에도 결측치가 남아 있는지 확인
remaining_missing = x[columns_fill_knn].isna().sum()
print("KNN Imputer 후 결측치 상태:")
print(remaining_missing)

# KNN Imputer 후에도 결측치가 여전히 존재한다면, 평균값이나 중앙값으로 채우기
for column in columns_fill_knn:
    if x[column].isna().sum() > 0:
        print(f"'{column}' 컬럼에서 결측치를 채우는 중...")
        # 평균값으로 채우기 (전용면적, 방수 등 수치형 데이터에 적합)
        x[column].fillna(x[column].mean(), inplace=True)

# 결측치가 모두 처리되었는지 확인
print("\n결측치 처리 후 상태:")
print(x[columns_fill_knn].isna().sum())


KNN Imputer 후 결측치 상태:
해당층      0
총층       0
전용면적     0
방수       0
욕실수      0
총주차대수    0
dtype: int64

결측치 처리 후 상태:
해당층      0
총층       0
전용면적     0
방수       0
욕실수      0
총주차대수    0
dtype: int64


In [27]:
# IterativeImputer 사용을 위한 활성화
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# 수치형 데이터만 선택 (수치형 데이터가 아닌 경우 제외)
numeric_columns = x[columns_fill_knn].select_dtypes(include=['float64', 'int64']).columns

# Iterative Imputer를 사용하여 결측치 채우기
iterative_imputer = IterativeImputer(max_iter=10, random_state=42)

# Iterative Imputer로 결측치를 채우기
x[numeric_columns] = iterative_imputer.fit_transform(x[numeric_columns])

# Iterative Imputer 후 결측치가 남아 있는지 확인
remaining_missing_iter = x[numeric_columns].isna().sum()
print("Iterative Imputer 후 결측치 상태:")
print(remaining_missing_iter)



Iterative Imputer 후 결측치 상태:
해당층      0
총층       0
전용면적     0
방수       0
욕실수      0
총주차대수    0
dtype: int64


In [28]:
train. describe()

,보증금,월세,전용면적,해당층,총층,방수,욕실수,총주차대수,관리비,허위매물여부
count,2.452000e+03,2452.000000,1665.000000,2223.000000,2436.000000,2436.000000,2434.000000,1756.000000,2452.000000,2452.000000
mean,1.574188e+08,380420.065253,27.009628,4.848403,8.052545,1.378079,1.034511,20.332574,5.477977,0.121533
std,1.212794e+08,206425.413601,7.951757,3.543348,4.920112,0.485007,0.182575,28.939900,5.631185,0.326813
min,5.000000e+06,0.000000,17.500000,1.000000,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,7.500000e+07,200000.000000,19.800000,2.000000,4.000000,1.000000,1.000000,4.000000,0.000000,0.000000
50%,1.325000e+08,380000.000000,26.400000,4.000000,6.000000,1.000000,1.000000,10.000000,5.000000,0.000000
75%,1.890000e+08,550000.000000,32.110000,6.000000,10.000000,2.000000,1.000000,31.000000,9.000000,0.000000
max,4.090000e+08,750000.000000,49.970000,21.000000,21.000000,2.000000,2.000000,590.000000,96.000000,1.000000


In [29]:
train. info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2452 entries, 0 to 2451
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      2452 non-null   object 
 1   매물확인방식  2452 non-null   object 
 2   보증금     2452 non-null   float64
 3   월세      2452 non-null   int64  
 4   전용면적    1665 non-null   float64
 5   해당층     2223 non-null   float64
 6   총층      2436 non-null   float64
 7   방향      2452 non-null   object 
 8   방수      2436 non-null   float64
 9   욕실수     2434 non-null   float64
 10  주차가능여부  2452 non-null   object 
 11  총주차대수   1756 non-null   float64
 12  관리비     2452 non-null   int64  
 13  중개사무소   2452 non-null   object 
 14  제공플랫폼   2452 non-null   object 
 15  게재일     2452 non-null   object 
 16  허위매물여부  2452 non-null   int64  
dtypes: float64(7), int64(3), object(7)
memory usage: 325.8+ KB


In [35]:
train. head()

,ID,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,총주차대수,관리비,중개사무소,제공플랫폼,게재일,허위매물여부
0,TRAIN_0000,현장확인,402500000.0,470000,17.500,9.6,15.0,서향,1.0,1.0,가능,40.0,96,t93Nt6I2I0,B플랫폼,2024-10-09,0
1,TRAIN_0001,현장확인,170500000.0,200000,36.334,3.0,4.0,남동향,2.0,1.0,불가능,2.4,0,q39iV5J4E6,D플랫폼,2024-12-26,0
2,TRAIN_0002,전화확인,114000000.0,380000,23.522,2.0,3.0,동향,1.0,1.0,불가능,16.2,0,b03oE4G3F6,A플랫폼,2024-11-28,0
3,TRAIN_0003,현장확인,163500000.0,30000,36.300,3.0,9.0,남동향,2.0,1.0,가능,13.0,10,G52Iz8V2B9,A플랫폼,2024-11-26,0
4,TRAIN_0004,현장확인,346000000.0,530000,38.260,3.0,3.0,동향,2.0,1.0,불가능,1.0,0,N45gM0M7R0,B플랫폼,2024-06-25,1


In [32]:
print(train.isna().sum())

ID          0
매물확인방식      0
보증금         0
월세          0
전용면적      787
해당층       229
총층         16
방향          0
방수         16
욕실수        18
주차가능여부      0
총주차대수     696
관리비         0
중개사무소       0
제공플랫폼       0
게재일         0
허위매물여부      0
dtype: int64


In [33]:
from sklearn.impute import KNNImputer

# 결측치를 채울 특성 선택
columns_fill_knn = ['전용면적', '해당층', '총층', '방수', '욕실수', '총주차대수']

# KNNImputer 설정 (n_neighbors=5로 설정)
knn_imputer = KNNImputer(n_neighbors=5)

# KNN Imputer로 결측치 채우기
train[columns_fill_knn] = knn_imputer.fit_transform(train[columns_fill_knn])

# 결측치 채운 후 확인
print(train[columns_fill_knn].isna().sum())

전용면적     0
해당층      0
총층       0
방수       0
욕실수      0
총주차대수    0
dtype: int64


In [36]:
## 2) Feature Engineering 추가

# 1 단위면적당 가격 (㎡당 가격)
x['단위면적당가격'] = x['보증금'] / x['전용면적']
x['단위면적당가격'].fillna(x['단위면적당가격'].median(), inplace=True)

# 2 보증금 대비 월세 비율
x['보증금_월세비율'] = x['보증금'] / (x['월세'] + 1)
x['보증금_월세비율'].fillna(x['보증금_월세비율'].median(), inplace=True)

# 3 층수 비율 (해당층 / 총층)
x['층수_비율'] = x['해당층'] / x['총층']
x['층수_비율'].fillna(x['층수_비율'].median(), inplace=True)

# 4 게재일 관련 Feature
x['게재일'] = pd.to_datetime(x['게재일'])
x['게재요일'] = x['게재일'].dt.weekday
x['게재일_경과일'] = (datetime(2025, 1, 20) - x['게재일']).dt.days

# 5 방향 그룹화
direction_map = {'동향': '동', '서향': '서', '남향': '남', '북향': '북', '남동향': '남', '북동향': '북'}
x['방향_그룹'] = x['방향'].map(direction_map)

# 6 이상 가격 탐지 Feature
unit_price_mean = x['단위면적당가격'].mean()
unit_price_std = x['단위면적당가격'].std()
x['가격_이상치'] = ((x['단위면적당가격'] - unit_price_mean) / unit_price_std).abs()

# 7 주차 가능 여부 수치 변환
x['주차가능여부'] = x['주차가능여부'].map({'가능': 1, '불가능': 0})

# 8 월세 + 관리비 총 비용
x['월세_총비용'] = x['월세'] + x['관리비']
x['월세_총비용'].fillna(x['월세_총비용'].median(), inplace=True)

# 9 관리비 비율 (관리비 / 월세)
x['관리비_비율'] = x['관리비'] / (x['월세'] + 1)
x['관리비_비율'].fillna(x['관리비_비율'].median(), inplace=True)

# 10 방수 밀집도 (방수 / 전용면적) & 욕실 밀집도 (욕실수 / 전용면적)
x['방수_밀집도'] = x['방수'] / (x['전용면적'] + 1)
x['욕실_밀집도'] = x['욕실수'] / (x['전용면적'] + 1)

# 11 플랫폼별 평균 보증금 / 월세 차이
플랫폼_보증금평균 = train.groupby('제공플랫폼')['보증금'].mean()
플랫폼_월세평균 = train.groupby('제공플랫폼')['월세'].mean()

x['제공플랫폼_보증금차이'] = x['보증금'] - x['제공플랫폼'].map(플랫폼_보증금평균)
x['제공플랫폼_월세차이'] = x['월세'] - x['제공플랫폼'].map(플랫폼_월세평균)

<ipython-input-36-84468a8c9509>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x['단위면적당가격'].fillna(x['단위면적당가격'].median(), inplace=True)
<ipython-input-36-84468a8c9509>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [116]:
## 3) Label Encoding (문자열 데이터를 숫자로 변환)
label_encode_cols = ['중개사무소', '게재일', '제공플랫폼', '방향', '방향_그룹']
label_encoders = {}
for col in label_encode_cols:
    le = LabelEncoder()
    x[col] = le.fit_transform(x[col].astype(str))
    label_encoders[col] = le  # 나중에 변환을 위해 저장

In [42]:
from sklearn.preprocessing import OneHotEncoder

one_hot_cols = ['매물확인방식', '주차가능여부']
one_hot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# 컬럼 존재 여부 확인 후 진행
if all(col in x.columns for col in one_hot_cols):
    x_encoded = one_hot_encoder.fit_transform(x[one_hot_cols])
    x_encoded_df = pd.DataFrame(x_encoded, columns=one_hot_encoder.get_feature_names_out(one_hot_cols), index=x.index)

    # 기존 데이터와 병합 후 기존 열 삭제
    x = pd.concat([x.drop(columns=one_hot_cols), x_encoded_df], axis=1)
else:
    print(f"컬럼 {one_hot_cols} 중 일부가 데이터프레임에 존재하지 않습니다.")


컬럼 ['매물확인방식', '주차가능여부'] 중 일부가 데이터프레임에 존재하지 않습니다.


In [41]:
print(x.columns)

Index(['보증금', '월세', '전용면적', '해당층', '총층', '방향', '방수', '욕실수', '총주차대수', '관리비',
       '중개사무소', '제공플랫폼', '게재일', '단위면적당가격', '보증금_월세비율', '층수_비율', '게재요일',
       '게재일_경과일', '방향_그룹', '가격_이상치', '월세_총비용', '관리비_비율', '방수_밀집도', '욕실_밀집도',
       '제공플랫폼_보증금차이', '제공플랫폼_월세차이', '매물확인방식_서류확인', '매물확인방식_전화확인',
       '매물확인방식_현장확인', '주차가능여부_0', '주차가능여부_1'],
      dtype='object')


In [47]:
# 원핫 인코딩 후 결측치 채우기 (최빈값으로 채우기)
x['매물확인방식_서류확인'].fillna(x['매물확인방식_서류확인'].mode()[0], inplace=True)
x['매물확인방식_전화확인'].fillna(x['매물확인방식_전화확인'].mode()[0], inplace=True)
x['매물확인방식_현장확인'].fillna(x['매물확인방식_현장확인'].mode()[0], inplace=True)
x['주차가능여부_0'].fillna(x['주차가능여부_0'].mode()[0], inplace=True)
x['주차가능여부_1'].fillna(x['주차가능여부_1'].mode()[0], inplace=True)

# 결측치가 잘 채워졌는지 확인
print(x.isna().sum())

print(x.head())


보증금            0
월세             0
전용면적           0
해당층            0
총층             0
방향             0
방수             0
욕실수            0
총주차대수          0
관리비            0
중개사무소          0
제공플랫폼          0
게재일            0
단위면적당가격        0
보증금_월세비율       0
층수_비율          0
게재요일           0
게재일_경과일        0
방향_그룹          0
가격_이상치         0
월세_총비용         0
관리비_비율         0
방수_밀집도         0
욕실_밀집도         0
제공플랫폼_보증금차이    0
제공플랫폼_월세차이     0
매물확인방식_서류확인    0
매물확인방식_전화확인    0
매물확인방식_현장확인    0
주차가능여부_0       0
주차가능여부_1       0
dtype: int64
           보증금      월세    전용면적  해당층    총층  방향   방수  욕실수  총주차대수  관리비  ...  \
0  402500000.0  470000  17.500  9.6  15.0   7  1.0  1.0   40.0   96  ...   
1  170500000.0  200000  36.334  3.0   4.0   0  2.0  1.0    2.4    0  ...   
2  114000000.0  380000  23.522  2.0   3.0   3  1.0  1.0   16.2    0  ...   
3  163500000.0   30000  36.300  3.0   9.0   0  2.0  1.0   13.0   10  ...   
4  346000000.0  530000  38.260  3.0   3.0   3  2.0  1.0    1.0    0  ...   

   

<ipython-input-47-588e3af346bf>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x['매물확인방식_서류확인'].fillna(x['매물확인방식_서류확인'].mode()[0], inplace=True)
<ipython-input-47-588e3af346bf>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace

In [46]:
print(x.columns)

Index(['보증금', '월세', '전용면적', '해당층', '총층', '방향', '방수', '욕실수', '총주차대수', '관리비',
       '중개사무소', '제공플랫폼', '게재일', '단위면적당가격', '보증금_월세비율', '층수_비율', '게재요일',
       '게재일_경과일', '방향_그룹', '가격_이상치', '월세_총비용', '관리비_비율', '방수_밀집도', '욕실_밀집도',
       '제공플랫폼_보증금차이', '제공플랫폼_월세차이', '매물확인방식_서류확인', '매물확인방식_전화확인',
       '매물확인방식_현장확인', '주차가능여부_0', '주차가능여부_1'],
      dtype='object')


In [50]:
# 5) Train / Validation 분할 (Stratified 방식)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

In [51]:
def add_noise(df, noise_level=0.05):
    numeric_cols = df.select_dtypes(include=[np.number]).columns  # 숫자형 컬럼만 선택
    df[numeric_cols] = df[numeric_cols] * (1 + noise_level * np.random.randn(*df[numeric_cols].shape))
    return df

print("Before Noise Injection:", len(x_train))
x_train = add_noise(x_train)
print("After Noise Injection:", len(x_train))

Before Noise Injection: 1961
After Noise Injection: 1961


In [52]:
def augment_minority_class(x_train, y_train, num_augments=2, noise_level=0.05):
    numeric_cols = x_train.select_dtypes(include=[np.number]).columns
    x_train_1 = x_train[y_train == 1].copy()  # 소수 클래스(1)만 선택
    augmented_data = []

    for _ in range(num_augments):
        x_aug = x_train_1.copy()
        x_aug[numeric_cols] = x_aug[numeric_cols] * (1 + noise_level * np.random.randn(*x_aug[numeric_cols].shape))
        augmented_data.append(x_aug)

    x_train = pd.concat([x_train] + augmented_data, axis=0).reset_index(drop=True)
    y_train = np.concatenate([y_train] + [np.ones(len(x_train_1))] * num_augments)  # 레이블 추가

    return x_train, y_train

print("Before Minority Augmentation:", len(x_train))
x_train, y_train = augment_minority_class(x_train, y_train, num_augments=2)
print("After Minority Augmentation:", len(x_train))

Before Minority Augmentation: 1961
After Minority Augmentation: 2437


In [55]:
# XGBoost 모델 정의 + 불균형 보정 옵션 적용
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])  # 0의 개수 / 1의 개수 

# XGBClassifier 객체를 명확하게 정의
xgb_model = XGBClassifier(scale_pos_weight=scale_pos_weight)

In [56]:
# GridSearchCV 설정
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [6, 10],
    'learning_rate': [0.01, 0.1],
    'gamma': [0, 0.1, 0.5]
}

grid_search = GridSearchCV(
    estimator=xgb_model,  # ✅ 명확하게 정의한 모델을 전달
    param_grid=param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(x_train, y_train)

# 6. 최적의 하이퍼파라미터 출력
print("Best parameters:", grid_search.best_params_)
print("Best Macro F1-score:", grid_search.best_score_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best parameters: {'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200}
Best Macro F1-score: 0.9633874601306459


In [57]:
# 최적 모델 학습 및 평가
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(x_val)

macro_f1 = f1_score(y_val, y_val_pred, average='macro')
print(f"Validation Macro F1-score: {macro_f1:.4f}")

Validation Macro F1-score: 0.8966


In [61]:
# Test 데이터 로드
test = pd.read_csv(f'{path}/test.csv')

In [63]:
# Test 결측값 대체
test[columns_fill_knn] = knn_imputer.transform(test[columns_fill_knn])

In [64]:
def create_new_features(x): 
    # 1 단위면적당 가격 (㎡당 가격)
    x['단위면적당가격'] = x['보증금'] / x['전용면적']
    x['단위면적당가격'].fillna(x['단위면적당가격'].median(), inplace=True)
    
    # 2 보증금 대비 월세 비율
    x['보증금_월세비율'] = x['보증금'] / (x['월세'] + 1)
    x['보증금_월세비율'].fillna(x['보증금_월세비율'].median(), inplace=True)
    
    # 3 층수 비율 (해당층 / 총층)
    x['층수_비율'] = x['해당층'] / x['총층']
    x['층수_비율'].fillna(x['층수_비율'].median(), inplace=True)
    
    # 4 게재일 관련 Feature
    x['게재일'] = pd.to_datetime(x['게재일'])
    x['게재요일'] = x['게재일'].dt.weekday
    x['게재일_경과일'] = (datetime(2025, 1, 20) - x['게재일']).dt.days
    
    # 5 방향 그룹화
    direction_map = {'동향': '동', '서향': '서', '남향': '남', '북향': '북', '남동향': '남', '북동향': '북'}
    x['방향_그룹'] = x['방향'].map(direction_map)
    
    # 6 이상 가격 탐지 Feature
    unit_price_mean = x['단위면적당가격'].mean()
    unit_price_std = x['단위면적당가격'].std()
    x['가격_이상치'] = ((x['단위면적당가격'] - unit_price_mean) / unit_price_std).abs()
    
    # 7 주차 가능 여부 수치 변환
    x['주차가능여부'] = x['주차가능여부'].map({'가능': 1, '불가능': 0})
    
    # 8 월세 + 관리비 총 비용
    x['월세_총비용'] = x['월세'] + x['관리비']
    x['월세_총비용'].fillna(x['월세_총비용'].median(), inplace=True)
    
    # 9 관리비 비율 (관리비 / 월세)
    x['관리비_비율'] = x['관리비'] / (x['월세'] + 1)
    x['관리비_비율'].fillna(x['관리비_비율'].median(), inplace=True)
    
    # 10 방수 밀집도 (방수 / 전용면적) & 욕실 밀집도 (욕실수 / 전용면적)
    x['방수_밀집도'] = x['방수'] / (x['전용면적'] + 1)
    x['욕실_밀집도'] = x['욕실수'] / (x['전용면적'] + 1)
    
    # 11 플랫폼별 평균 보증금 / 월세 차이
    플랫폼_보증금평균 = train.groupby('제공플랫폼')['보증금'].mean()
    플랫폼_월세평균 = train.groupby('제공플랫폼')['월세'].mean()
    
    x['제공플랫폼_보증금차이'] = x['보증금'] - x['제공플랫폼'].map(플랫폼_보증금평균)
    x['제공플랫폼_월세차이'] = x['월세'] - x['제공플랫폼'].map(플랫폼_월세평균)
    return x

test = create_new_features(test)
test.head()

<ipython-input-64-f15887735e83>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x['단위면적당가격'].fillna(x['단위면적당가격'].median(), inplace=True)
<ipython-input-64-f15887735e83>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

,ID,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,...,게재요일,게재일_경과일,방향_그룹,가격_이상치,월세_총비용,관리비_비율,방수_밀집도,욕실_밀집도,제공플랫폼_보증금차이,제공플랫폼_월세차이
0,TEST_000,서류확인,223000000.0,530000,29.912,5.0,5.0,남향,1.0,1.0,...,0,91,남,0.372327,530007,0.000013,0.032350,0.032350,7.031793e+07,150326.086957
1,TEST_001,서류확인,150500000.0,590000,30.260,7.0,11.0,서향,1.0,1.0,...,5,506,서,0.162124,590011,0.000019,0.031990,0.031990,-2.182065e+06,210326.086957
2,TEST_002,현장확인,47000000.0,200000,41.500,2.0,3.0,남서향,2.0,1.0,...,2,292,NaN,0.989335,200000,0.000000,0.047059,0.023529,-1.094016e+08,-190639.344262
3,TEST_003,서류확인,133000000.0,250000,31.350,5.0,6.0,남향,2.0,1.0,...,3,123,남,0.319583,250005,0.000020,0.061824,0.030912,-2.340164e+07,-140639.344262
4,TEST_004,현장확인,108000000.0,380000,23.140,2.0,3.0,남향,1.0,1.0,...,6,71,남,0.228093,380000,0.000000,0.041425,0.041425,-4.683716e+07,1649.269311


In [77]:
# 학습된 인코더의 범주 확인
for col, categories in zip(one_hot_cols, one_hot_encoder.categories_):
    print(f"Column: {col}, Categories in encoder: {categories}")

# 테스트 데이터에서 범주 확인
for col in one_hot_cols:
    print(f"Column: {col}, Unique values in test: {test[col].unique()}")




Column: 매물확인방식, Categories in encoder: ['서류확인' '전화확인' '현장확인']
Column: 주차가능여부, Categories in encoder: ['가능' '불가능']
Column: 매물확인방식, Unique values in test: ['서류확인', '현장확인', '전화확인']
Categories (3, object): ['서류확인', '전화확인', '현장확인']
Column: 주차가능여부, Unique values in test: [0, 1]
Categories (2, int64): [0, 1]


In [79]:
test['주차가능여부'] = test['주차가능여부'].replace({1: '가능', 0: '불가능'})

<ipython-input-79-2545f0ac423c>:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  test['주차가능여부'] = test['주차가능여부'].replace({1: '가능', 0: '불가능'})


In [80]:
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)


In [83]:
if 'ID' in test.columns:
    test.drop(columns=['ID'], inplace=True)
else:
    print("컬럼 'ID'가 존재하지 않습니다.")

컬럼 'ID'가 존재하지 않습니다.


In [86]:
test.drop(columns=['ID'], inplace=True, errors='ignore')

In [94]:
print(test.columns)

Index(['매물확인방식', '보증금', '월세', '전용면적', '해당층', '총층', '방향', '방수', '욕실수', '주차가능여부',
       '총주차대수', '관리비', '중개사무소', '제공플랫폼', '게재일', '단위면적당가격', '보증금_월세비율', '층수_비율',
       '게재요일', '게재일_경과일', '방향_그룹', '가격_이상치', '월세_총비용', '관리비_비율', '방수_밀집도',
       '욕실_밀집도', '제공플랫폼_보증금차이', '제공플랫폼_월세차이'],
      dtype='object')


In [95]:
test.head()

,매물확인방식,보증금,월세,전용면적,해당층,총층,방향,방수,욕실수,주차가능여부,...,게재요일,게재일_경과일,방향_그룹,가격_이상치,월세_총비용,관리비_비율,방수_밀집도,욕실_밀집도,제공플랫폼_보증금차이,제공플랫폼_월세차이
0,서류확인,223000000.0,530000,29.912,5.0,5.0,22,1.0,1.0,불가능,...,0,91,11,0.372327,530007,0.000013,0.032350,0.032350,7.031793e+07,150326.086957
1,서류확인,150500000.0,590000,30.260,7.0,11.0,17,1.0,1.0,불가능,...,5,506,10,0.162124,590011,0.000019,0.031990,0.031990,-2.182065e+06,210326.086957
2,현장확인,47000000.0,200000,41.500,2.0,3.0,21,2.0,1.0,불가능,...,2,292,14,0.989335,200000,0.000000,0.047059,0.023529,-1.094016e+08,-190639.344262
3,서류확인,133000000.0,250000,31.350,5.0,6.0,22,2.0,1.0,가능,...,3,123,11,0.319583,250005,0.000020,0.061824,0.030912,-2.340164e+07,-140639.344262
4,현장확인,108000000.0,380000,23.140,2.0,3.0,22,1.0,1.0,불가능,...,6,71,11,0.228093,380000,0.000000,0.041425,0.041425,-4.683716e+07,1649.269311


In [98]:
train_features = best_model.get_booster().feature_names
test_features = test.columns

missing_features = set(train_features) - set(test_features)
for feature in missing_features:
    test[feature] = 0


In [102]:
print(test.columns)

Index(['보증금', '월세', '전용면적', '해당층', '총층', '방향', '방수', '욕실수', '총주차대수', '관리비',
       '중개사무소', '제공플랫폼', '게재일', '단위면적당가격', '보증금_월세비율', '층수_비율', '게재요일',
       '게재일_경과일', '방향_그룹', '가격_이상치', '월세_총비용', '관리비_비율', '방수_밀집도', '욕실_밀집도',
       '제공플랫폼_보증금차이', '제공플랫폼_월세차이', '주차가능여부_1', '주차가능여부_0', '매물확인방식_서류확인',
       '매물확인방식_현장확인', '매물확인방식_전화확인'],
      dtype='object')


In [103]:
print(test.columns)

Index(['보증금', '월세', '전용면적', '해당층', '총층', '방향', '방수', '욕실수', '총주차대수', '관리비',
       '중개사무소', '제공플랫폼', '게재일', '단위면적당가격', '보증금_월세비율', '층수_비율', '게재요일',
       '게재일_경과일', '방향_그룹', '가격_이상치', '월세_총비용', '관리비_비율', '방수_밀집도', '욕실_밀집도',
       '제공플랫폼_보증금차이', '제공플랫폼_월세차이', '주차가능여부_1', '주차가능여부_0', '매물확인방식_서류확인',
       '매물확인방식_현장확인', '매물확인방식_전화확인'],
      dtype='object')


In [105]:
test.rename(columns={'old_column_name': '매물확인방식', 'another_old_name': '주차가능여부'}, inplace=True)


In [107]:
print(test.columns)
print(best_model.get_booster().feature_names)

Index(['보증금', '월세', '전용면적', '해당층', '총층', '방향', '방수', '욕실수', '총주차대수', '관리비',
       '중개사무소', '제공플랫폼', '게재일', '단위면적당가격', '보증금_월세비율', '층수_비율', '게재요일',
       '게재일_경과일', '방향_그룹', '가격_이상치', '월세_총비용', '관리비_비율', '방수_밀집도', '욕실_밀집도',
       '제공플랫폼_보증금차이', '제공플랫폼_월세차이', '주차가능여부_1', '주차가능여부_0', '매물확인방식_서류확인',
       '매물확인방식_현장확인', '매물확인방식_전화확인'],
      dtype='object')
['보증금', '월세', '전용면적', '해당층', '총층', '방향', '방수', '욕실수', '총주차대수', '관리비', '중개사무소', '제공플랫폼', '게재일', '단위면적당가격', '보증금_월세비율', '층수_비율', '게재요일', '게재일_경과일', '방향_그룹', '가격_이상치', '월세_총비용', '관리비_비율', '방수_밀집도', '욕실_밀집도', '제공플랫폼_보증금차이', '제공플랫폼_월세차이', '매물확인방식_서류확인', '매물확인방식_전화확인', '매물확인방식_현장확인', '주차가능여부_0', '주차가능여부_1']


In [110]:
# 모델이 기대하는 훈련 데이터의 특성 순서를 가져옵니다.
train_features = best_model.get_booster().feature_names

# 테스트 데이터의 특성을 훈련 데이터와 동일한 순서로 정렬합니다.
test = test[train_features]

# 예측 수행
pred = pd.Series(best_model.predict(test))


In [111]:
# 결측치 확인
print(test.isnull().sum())

# 결측치가 있다면 0으로 대체 (또는 다른 적절한 방식으로 처리)
test.fillna(0, inplace=True)


보증금            0
월세             0
전용면적           0
해당층            0
총층             0
방향             0
방수             0
욕실수            0
총주차대수          0
관리비            0
중개사무소          0
제공플랫폼          0
게재일            0
단위면적당가격        0
보증금_월세비율       0
층수_비율          0
게재요일           0
게재일_경과일        0
방향_그룹          0
가격_이상치         0
월세_총비용         0
관리비_비율         0
방수_밀집도         0
욕실_밀집도         0
제공플랫폼_보증금차이    1
제공플랫폼_월세차이     1
매물확인방식_서류확인    0
매물확인방식_전화확인    0
매물확인방식_현장확인    0
주차가능여부_0       0
주차가능여부_1       0
dtype: int64


In [ ]:
print(test.dtypes)  # 데이터 타입 출력

# float32 또는 float64로 변환
test = test.astype('float32')


In [113]:
# 훈련 데이터 특성 순서에 맞게 테스트 데이터 정렬
train_features = best_model.get_booster().feature_names
test = test[train_features]

# 결측치 처리
test.fillna(0, inplace=True)

# 데이터 타입 변환 (필요시)
test = test.astype('float32')

# 예측 수행
pred = pd.Series(best_model.predict(test))


In [124]:
import os

# 디렉토리 내의 파일 목록을 확인
path = "/kaggle/input/open12"
files = os.listdir(path)
print("Files in directory:", files)

Files in directory: ['sample_submission.csv', 'train.csv', 'test.csv']


In [125]:
import pandas as pd

# 경로에 있는 sample_submission.csv 파일을 pandas로 읽기
file_path = '/kaggle/input/open12/sample_submission.csv'

try:
    submit = pd.read_csv(file_path)
    print(submit.head())
except Exception as e:
    print(f"Error: {e}")


         ID  허위매물여부
0  TEST_000       0
1  TEST_001       0
2  TEST_002       0
3  TEST_003       0
4  TEST_004       0


In [126]:
submit['허위매물여부'] = pred # 우리의 예측 넣는다
submit.head()

,ID,허위매물여부
0,TEST_000,0
1,TEST_001,1
2,TEST_002,1
3,TEST_003,0
4,TEST_004,0


In [127]:
submit.to_csv('./baseline_submission.csv',index=False)